# Stroke dataset prediction

## Contexto:

Según la Organización Mundial de la Salud (OMS), el accidente cerebrovascular es la segunda causa principal de muerte a nivel mundial, responsable de aproximadamente el 11% del total de muertes.
Este conjunto de datos se utiliza para predecir si es probable que un paciente sufra un accidente cerebrovascular en función de los parámetros de entrada como el sexo, la edad, diversas enfermedades y el tabaquismo. Cada fila de los datos proporciona información relevante sobre el paciente.

Informacion de los atributos
* id: identificador unico
* gender: "Male", "Female" o "Other"
* age: Edad del paciente
* hypertension: 0 si el paciente no tiene hipertension, 1 si el paciente tiene hipertension
* heart_disease:0 si el paciente no tiene ninguna enfermedad cardíaca, 1 si el paciente tiene una enfermedad cardíaca
* ever_married: "No" o "Yes"
* work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
* Residence_type: "Rural" or "Urban"
* avg_glucose_level: nivel medio de glucosa en sangre
* bmi: indice de masa corporal
* smoking_status: "formerly smoked", "never smoked", "smokes" o "Unknown"*
* stroke: accidente cerebrovascular: 1 si el paciente tuvo un accidente cerebrovascular o 0 si no

*Note: "Unknown" en smoking_status significa que la información no está disponible para este paciente.

https://www.kaggle.com/fedesoriano/stroke-prediction-dataset

Antes de continuar con el análisis se recomienda leer el artículo [[1]](https://medlineplus.gov/spanish/ency/article/000726.htm#:~:text=La%20presi%C3%B3n%20arterial%20muy%20alta,en%20un%20accidente%20cerebrovascular%20hemorr%C3%A1gico.) sobre Accidente Cerebrovascular con objetivo de familiarizarse con el problema y entender las hipótesis que se plantearan a continuación:

Segun [[1]](https://medlineplus.gov/spanish/ency/article/000726.htm#:~:text=La%20presi%C3%B3n%20arterial%20muy%20alta,en%20un%20accidente%20cerebrovascular%20hemorr%C3%A1gico.), "La presión arterial alta es el principal factor de riesgo para los accidentes cerebrovasculares". Entre otros factores de riesgo de nuestro interes se mencionan los siguientes:

* Diabetes
* Ser hombre
* Aumento de la edad, especialmente después de los 55 años

En [[1]](https://medlineplus.gov/spanish/ency/article/000726.htm#:~:text=La%20presi%C3%B3n%20arterial%20muy%20alta,en%20un%20accidente%20cerebrovascular%20hemorr%C3%A1gico.) tambien se denota que el riesgo de accidente cerebrovascular es también mayor en:

* Personas que tienen una enfermedad cardíaca o mala circulación en las piernas causada por estrechamiento de las arterias
* Personas que tienen hábitos de un estilo de vida malsano tales como el tabaquismo, consumo excesivo de alcohol, consumo de drogas, una dieta rica en grasa y falta de ejercicio
* Mujeres que toman píldoras anticonceptivas (especialmente las que fuman y son mayores de 35 años)

## Hipótesis principales
En este documento buscaremos comprobar las siguientes hipotesis:
1. El ACV es mas frecuente en personas mayores a 55 años
2. Los niveles de glucosa son mas altos en personas que padecieron un ACV
3. Los hombres son mas propensos a padecer accidente cerebrovascular
4. La hipertensión esta guarda una estrecha relación con el ACV
5. Las enfermedades cardíacas guardan estrecha relación con el ACV


Así mismo se analizarán otras variables de interés dadas por el dataset de manera complementaria.

## Análisis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import plotly.figure_factory as ff

In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
print(df.shape)
df.head()

In [ ]:
df[['age','avg_glucose_level','bmi']].describe()

### ¿Qué porcentaje de las personas sufrieron un ACV?

In [ ]:
counts = df['stroke'].value_counts().reset_index()
counts['index'] = np.where(counts['index'] == 0, 'No stroke','stroke')

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "xy"}, {"type": "domain"}]],
    subplot_titles=("Personas con ACV vs las que no", "Porcentaje de víctimas"))


fig.add_trace(go.Bar(x=counts['index'], y=counts['stroke']),
              row=1, col=1)

fig.update_xaxes(title_text="Sufrio ACV", row=1, col=1)
fig.update_yaxes(title_text="Cantidad", row=1, col=1)



fig.add_trace(go.Pie(values=counts['stroke'], labels=counts['index'], textinfo='label+percent'),
              row=1, col=2)


fig.update_layout(title='Distribucion de la variable objetivo', height=500, showlegend=False)

fig.show()

### ¿Que diferencias hay entre las victimas y las personas sanas?

In [ ]:
df_no_stroke = df[df['stroke'] == 0]
df_stroke = df[df['stroke'] == 1]

In [ ]:
df_no_stroke[['age','avg_glucose_level','bmi']].describe()

In [ ]:
df_stroke[['age','avg_glucose_level','bmi']].describe()

A simple vista puede notarse que las personas que sufrieron un ACV tienen una mayor media y mediana de edades que las que no.
Lo mismo se cumple, en menor medida, para el promedio de glucosa en sangre.

### Edad

In [ ]:
hist_data = [df_stroke['age'], df_no_stroke['age']]

group_labels = ['stroke age', 'no stroke age']
colors = ['#A56CC1', '#63F5EF']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=2, colors=colors, show_rug=False)
fig.update_layout(title_text='Edades de víctimas y no víctimas')

fig.show()

El grafico muestra la distribucion de edades de las personas que padecieron un ACV y de las que no. Vemos que la mayoria de personas que padecieron un accidente cerebrovascular tienen edades mayores a 55 años, tal y como se menciona en [[1]](https://medlineplus.gov/spanish/ency/article/000726.htm#:~:text=La%20presi%C3%B3n%20arterial%20muy%20alta,en%20un%20accidente%20cerebrovascular%20hemorr%C3%A1gico.) lo cual valida nuestra primer hipotesis (el ACV es mas frecuente en personas mayores a 55 años).

### Nivel promedio de glucosa en sangre

In [ ]:
hist_data = [df_stroke['avg_glucose_level'], df_no_stroke['avg_glucose_level']]

group_labels = ['stroke avg_glucose_level', 'no stroke avg_glucose_level']
colors = ['#A56CC1', '#63F5EF']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=10, colors=colors, show_rug=False)
fig.update_layout(title_text='Niveles de glucosa de de víctimas y no víctimas')

fig.show()

Según [[1]](https://medlineplus.gov/spanish/ency/article/000726.htm#:~:text=La%20presi%C3%B3n%20arterial%20muy%20alta,en%20un%20accidente%20cerebrovascular%20hemorr%C3%A1gico.), la diabetes es un factor de riesgo importante para el accidente cerebrovascular. Aquí vemos que las personas víctimas de ACV tienen niveles de glucosa mas altos que las personas que no, lo cual valida nuestra segunda hipótesis.

In [ ]:
hist_data = [df_stroke['bmi'].dropna(), df_no_stroke['bmi'].dropna()]

group_labels = ['stroke bmi', 'no stroke bmi']
colors = ['#A56CC1', '#63F5EF']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=2, colors=colors, show_rug=False)
fig.update_layout(title_text='Bmi de víctimas y no víctimas')

fig.show()

Por otra parte, el índice de masa corporal pareciera distribuirse casi de igual manera en ambos grupos.

### Genero

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Víctimas de ACV", 
                    "No víctimas de ACV")
)

group_gender = df_stroke.groupby(['gender'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_gender['stroke'], labels=group_gender['gender'], textinfo='label+percent'),
              row=1, col=1)

group_gender = df_no_stroke.groupby(['gender'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_gender['stroke'], labels=group_gender['gender'], textinfo='label+percent'),
              row=1, col=2)

fig.update_layout(title='Generos en', height=500)

fig.show()

Segun el gráfico, no pareciera haber una predominancia de algun género entre las personas que sufrieron un ACV y las que no. Lo cual se contradice con nuestra tercer hipotesis: "los hombres son mas propensos a padecer accidente cerebrovascular". De hecho el porcentaje de mujeres es mayor entre las víctimas.

### Hipertensión

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Víctimas de ACV",
                    "No víctimas de ACV")
)

group_hypertension = df_stroke.groupby(['hypertension'])['stroke'].count().reset_index()
group_hypertension['hypertension'] = np.where(group_hypertension['hypertension'] == 1, 'Hypertension', 'without hypertension')
fig.add_trace(go.Pie(values=group_hypertension['stroke'], labels=group_hypertension['hypertension'], hole=.3, textinfo='label+percent'),
              row=1, col=1)

group_hypertension = df_no_stroke.groupby(['hypertension'])['stroke'].count().reset_index()
group_hypertension['hypertension'] = np.where(group_hypertension['hypertension'] == 1, 'Hypertension', 'without hypertension')
fig.add_trace(go.Pie(values=group_hypertension['stroke'], labels=group_hypertension['hypertension'], hole=.3, textinfo='label+percent'),
              row=1, col=2)

fig.update_layout(title='Hiptertensión en', height=500)

fig.show()

Notamos que el porcentaje de personas que padecen hipertensión es mayor en aquellas que tuvieron un accidente cerebrovascular que en las que nunca tuvieron este tipo de problema. Según 
[[1]](https://medlineplus.gov/spanish/ency/article/000726.htm#:~:text=La%20presi%C3%B3n%20arterial%20muy%20alta,en%20un%20accidente%20cerebrovascular%20hemorr%C3%A1gico.) "La presión arterial alta es el principal factor de riesgo para los accidentes cerebrovasculares.".

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Con hipertensión", 
                    "Sin hipertensión")
)

df_ht = df[df['hypertension'] == 1]
df_no_ht = df[df['hypertension'] == 0]


group_hypertension = df_ht.groupby('stroke').agg({'stroke':'count'}).rename(columns={'stroke':'count'}).reset_index()
group_hypertension['stroke'] = np.where(group_hypertension['stroke'] == 1, 'Stroke', 'No stroke')
fig.add_trace(go.Pie(values=group_hypertension['count'], labels=group_hypertension['stroke'], hole=0.3),
              row=1, col=1)

group_hypertension = df_no_ht.groupby('stroke').agg({'stroke':'count'}).rename(columns={'stroke':'count'}).reset_index()
group_hypertension['stroke'] = np.where(group_hypertension['stroke'] == 1, 'Stroke', 'No stroke')
fig.add_trace(go.Pie(values=group_hypertension['count'], labels=group_hypertension['stroke'], hole=0.3),
              row=1, col=2)

fig.update_layout(title='ACV en personas', height=500)

fig.show()

Visto desde otro ángulo, el porcentaje de victimas de accidente cerebrovascular es mayor (13%) en las personas que padecen hipertensión que en las que no (3%).

### Enfermedades cardíacas

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Víctimas de ACV", 
                    "No víctimas de ACV")
)

group_heart_disease = df_stroke.groupby(['heart_disease'])['stroke'].count().reset_index()
group_heart_disease['heart_disease'] = np.where(group_heart_disease['heart_disease'] == 1,'heart disease','without heart disease')
fig.add_trace(go.Pie(values=group_heart_disease['stroke'], labels=group_heart_disease['heart_disease']),
              row=1, col=1)

group_heart_disease = df_no_stroke.groupby(['heart_disease'])['stroke'].count().reset_index()
group_heart_disease['heart_disease'] = np.where(group_heart_disease['heart_disease'] == 1,'heart disease','without heart disease')
fig.add_trace(go.Pie(values=group_heart_disease['stroke'], labels=group_heart_disease['heart_disease']),
              row=1, col=2)

fig.update_layout(title='Enfermedades cardíacas en', height=500)

fig.show()

En las enfermedades cardíacas vemos el mismo escenario que en la hipertensión. El conjunto de personas víctimas de ACV contiene un mayor porcentaje con problemas cardíacos que las personas que no, lo que verifica una de nuestras hipótesis iniciales: "Las enfermedades cardíacas guardan estrecha relacion con el ACV"

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Con enfermedades cardícacas", 
                    "Sin enfermedades cardícacas")
)

df_hd = df[df['heart_disease'] == 1]
df_no_hd = df[df['heart_disease'] == 0]


group_heart = df_hd.groupby('stroke').agg({'stroke':'count'}).rename(columns={'stroke':'count'}).reset_index()
group_heart['stroke'] = np.where(group_heart['stroke'] == 1, 'Stroke', 'No stroke')
fig.add_trace(go.Pie(values=group_heart['count'], labels=group_heart['stroke'], hole=0.3),
              row=1, col=1)

group_heart = df_no_ht.groupby('stroke').agg({'stroke':'count'}).rename(columns={'stroke':'count'}).reset_index()
group_heart['stroke'] = np.where(group_heart['stroke'] == 1, 'Stroke', 'No stroke')
fig.add_trace(go.Pie(values=group_heart['count'], labels=group_heart['stroke'], hole=0.3),
              row=1, col=2)

fig.update_layout(title='ACV en personas', height=500)

fig.show()

Nuevamente, al observar los datos desde otro punto de vista, el porcentaje de personas que padecieron un acv es mayor en aquellas que tienen algun tipo de problema cardíaco que en las que no.

## Otras variables de interes:

### Matrimonio

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Víctimas de ACV", 
                    "No víctimas de ACV")
)

group_ever_married = df_stroke.groupby(['ever_married'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_ever_married['stroke'], labels=group_ever_married['ever_married'], textinfo='label+percent'),
              row=1, col=1)

group_ever_married = df_no_stroke.groupby(['ever_married'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_ever_married['stroke'], labels=group_ever_married['ever_married'], textinfo='label+percent'),
              row=1, col=2)

fig.update_layout(title='Matrimonio en', height=500)

fig.show()

Podemos observar que el porcentaje de personas casadas es notablemete superior entre las víctimas de ACV (aproximadamente en un 20%). Así mismo, no se tiene información suficiente para asegurar que ambas variables guardan algún tipo de relación.

### Tipo de trabajo

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Víctimas de ACV", 
                    "No víctimas de ACV")
)

group_work_type = df_stroke.groupby(['work_type'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_work_type['stroke'], labels=group_work_type['work_type'], textinfo='label+percent'),
              row=1, col=1)

group_work_type = df_no_stroke.groupby(['work_type'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_work_type['stroke'], labels=group_work_type['work_type'], textinfo='label+percent'),
              row=1, col=2)

fig.update_layout(title='Tipo de trabajo en', height=500)

fig.show()

Los tipos de trabajos se distribuyen de igual manera en ambos grupos, por lo que no pareciera ser una variable relevante a tener en cuenta.

### Fumar influye?

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Víctimas de ACV", 
                    "No víctimas de ACV")
)

group_smoking_status = df_stroke.groupby(['smoking_status'])['stroke'].count().reset_index()
group_smoking_status = group_smoking_status[group_smoking_status['smoking_status'] != 'Unknown']
fig.add_trace(go.Pie(values=group_smoking_status['stroke'], labels=group_smoking_status['smoking_status'], textinfo='label+percent'),
              row=1, col=1)

group_smoking_status = df_no_stroke.groupby(['smoking_status'])['stroke'].count().reset_index()
group_smoking_status = group_smoking_status[group_smoking_status['smoking_status'] != 'Unknown']
fig.add_trace(go.Pie(values=group_smoking_status['stroke'], labels=group_smoking_status['smoking_status'], textinfo='label+percent'),
              row=1, col=2)

fig.update_layout(title='Tabaquismo en ', height=500)

fig.show()

De igual manera, según este gráfico pareciera no haber una clara relacion entre el fumar/no fumar con el accidente cerebrovascular.

### La residencia

In [ ]:
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}]],
    subplot_titles=("Víctimas de ACV", 
                    "No víctimas de ACV")
)

group_residence = df_stroke.groupby(['Residence_type'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_residence['stroke'], labels=group_residence['Residence_type'], hole=0.3, textinfo='label+percent'),
              row=1, col=1)

group_residence = df_no_stroke.groupby(['Residence_type'])['stroke'].count().reset_index()
fig.add_trace(go.Pie(values=group_residence['stroke'], labels=group_residence['Residence_type'], hole=0.3, textinfo='label+percent'),
              row=1, col=2)

fig.update_layout(title='Residencia en', height=500)

fig.show()

Nuevamente, tampoco pareciera haber una relación clara entre la residencia de la persona (urbana o rural) y la posibilidad de tener un ACV.

### Relaciones entre las variables continuas

Para solucionar el desbalance del dataset opté por balancearlo hacia la clase con menor cantidad de ocurrencias (stroke = 1), esto hará que se descarten la mayoría de los ejemplos, pero al menos nos quedarán aproximadamente 500 datos para entrenar, testear y graficar de manera mas simple.

Recomiendo ver el siguiente link para explorar y experimentar con otras posibles soluciones: https://www.machinecurve.com/index.php/2020/11/10/working-with-imbalanced-datasets-with-tensorflow-and-keras/

In [ ]:
print(df_stroke.shape)
print(df_no_stroke.shape)

df_no_stroke = df_no_stroke.sample(df_stroke.shape[0])
df = df_stroke.append(df_no_stroke)

print(df.shape)

Elimino nulos para poder graficar:)

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()

#### Promedio de glucosa en funcion de la edad

In [ ]:
fig = px.scatter(df, x="age", y="avg_glucose_level", color="avg_glucose_level", color_continuous_scale='bluered',
                 size='age', hover_data=['bmi'])
fig.show()

En el anterior gráfico puede apreciarse el nivel promedio de glucosa en sangre en función de la edad. No pareciera haber muchas personas menores a 35 años con niveles de glucosa superiores a 150 mg/dl. Así mismo, para edades superiores a 35 años se observan dos nubes de puntos, ubicandose una por encima de los 150 mg/dl llegando hasta los 250 mg/dl y otra por debajo ubicandose cerca de los 100 mg/dl

*Nota: En la info del dataset no explicita la unidad de medida del promedio de glucosa en sangre, por lo que me base en lo leido en este [artículo](https://medlineplus.gov/spanish/ency/patientinstructions/000966.htm) para colocar la unidad mg/dl.*

#### Índice de masa corporal en funcion del promedio de glucosa

In [ ]:
fig = px.scatter(df, x="avg_glucose_level", y="bmi", color="bmi", color_continuous_scale='bluered',
                 size='bmi', hover_data=['bmi'])
fig.show()

Al graficar el índice de masa corporal en funcion del promedio de glucosa en sangre vemos las mismas nubes de puntos del gráfico anterior. A su vez, vemos tambien que las personas que tienen mayor un promedio de glucosa en sangre tienden, ligeramente, a tener un mayor indice de masa corporal.

#### Índice de masa corporal en funcion de la edad

In [ ]:
fig = px.scatter(df, x="age", y="bmi", color="bmi", color_continuous_scale='bluered',
                 size='avg_glucose_level', hover_data=['bmi'])
fig.show()

Al observar como se distribuye el BMI en función de la edad, vemos que a partir de los 20 años (aproximadamente) la gran mayoria de ejemplos se encuentran entre 20 y 40, mientras que algunos outliers se encuentran por encima o por debajo.

### Edad, índice de masa corporal y promedio de glucosa en sangre

In [ ]:
fig = px.scatter_3d(df, x='age', y='avg_glucose_level', z='bmi',
              color='stroke', color_continuous_scale='bluered', opacity=0.5)
fig.show()

Por último en el gráfico 3D podemos observar como se relacionan estas tres variables entre si desde distintas perspectivas. Los puntos rojos representan a personas que tuvieron ACV mientras que los azules representan a las que no.

## Conclusiones
Luego de completar nuestro análisis exploratorio del conjunto de datos, podemos concluir que la edad es una variable altamente relacionada con la posibilidad de padecer un accidente cerbrovascular, puesto que la distribución de edades de las víctimas, se agrupa en torno a los 50-80 años de edad, contrariamente a la del resto de personas en las que se distribuye de manera uniforme. Así mismo, el nivel promedio de glucosa en sangre pareceria ser una variable de interés, ya que en las víctimas adopta valores superiores o inferiores al del resto de personas.

Por otra parte, tanto la hipertensión como las enfermedades cardíacas son signos de alerta en la predicción de un posible ACV, puesto que el porcentaje de victimas es mayor en las personas que padecen alguno de estos factores. Por último, los datos indican que no existe una predominancia sobre alguno de los sexos dentro del conjunto de personas víctimas de ACV.

## Pre-procesado de datos, entrenamiento y predicción

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import tensorflow as tf

Convierto a numeros cada una de las features categoricas que estaban en formato de texto

In [ ]:
df['ever_married'] = np.where(df['ever_married'] == 'Yes', 1, 0)
df['Residence_type'] = np.where(df['Residence_type'] == 'Urban', 1, 0)

work_type_map = {
    'Private': 1,
    'Self-employed': 2,
    'Govt_job': 3,
    'children': 4, 
    'Never_worked': 5, 
}

df['work_type'] = df['work_type'].map(work_type_map)

smoking_status_map = {
    'formerly smoked': 1,
    'never smoked': 2,
    'smokes': 3,
    'Unknown': 4,
}

df['smoking_status'] = df['smoking_status'].map(smoking_status_map)

gender_map = {
    'Male': 1,
    'Female': 2
}

df['gender'] = df['gender'].map(gender_map)

df.head()

Debido a que en la columna "gender" hay un solo registro con el tipo "others" se eliminará dicho ejemplo.

In [ ]:
print(df['gender'].value_counts())
print('='*63)

df = df[df['gender'] != 'Other']

print(df['gender'].value_counts())
print('='*63)

print(df.shape)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['stroke','id'], axis=1), 
                                                    df['stroke'], 
                                                    test_size=0.3,
                                                   random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)

Por último se escalarán las variables del dataset para que el modelo las aprenda mas facilmente: 
https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_train

In [ ]:
x_test = scaler.transform(x_test)
x_test

## Entrenamiento

Se construyó una red neuronal de tipo MLP como modelo predictivo. La misma cuenta con cuatro capas de 32, 16, 8 y 1 neuronas respectivamente. Todas las neuronas cuentan con función de activación relu, exceptuando la de la última capa, en la cual se utilizó función de activacion sigmoide para realizar la predicción final. Dada la naturaleza del problema, también se utilizaron las métricas Precision y Recall para evaluar la performance del modelo. Recomiendo ver el siguiente [artículo](https://www.tutorialspoint.com/machine_learning_with_python/machine_learning_algorithms_performance_metrics.htm) si se desea profundizar mas en el criterio de selección de métricas.

In [ ]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(32, input_dim = x_train.shape[1], activation="relu"))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(8,activation="relu"))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(1, activation = "sigmoid")) 

model.compile(optimizer='sgd',loss="binary_crossentropy",metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

print(model.summary())

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), 
          epochs=400, batch_size=32)

Luego de entrenar al modelo por 400 épocas, notamos que los valores de validación de precision y recall se encuentran por encima del 70%. Teniendo en cuenta que este modelo no se utilizará de manera productiva, considero que son niveles aceptables.

## Curvas de aprendizaje

In [ ]:
df_metrics = pd.DataFrame(model.history.history).reset_index().rename(columns={'index':'epoch'})
df_metrics

In [ ]:
df_metrics = pd.DataFrame(model.history.history)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_metrics.index, y=df_metrics['recall'],
                    mode='lines',
                    name='recall'))
fig.add_trace(go.Scatter(x=df_metrics.index, y=df_metrics['val_recall'],
                    mode='lines',
                    name='val_recall'))

fig.update_layout(title='Recall',
                   xaxis_title='Epoch',
                   yaxis_title='Recall')

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_metrics.index, y=df_metrics['precision'],
                    mode='lines',
                    name='precision'))
fig.add_trace(go.Scatter(x=df_metrics.index, y=df_metrics['val_precision'],
                    mode='lines',
                    name='val_precision'))

fig.update_layout(title='Precision',
                   xaxis_title='Epoch',
                   yaxis_title='Precision')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_metrics.index, y=df_metrics['loss'],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(x=df_metrics.index, y=df_metrics['val_loss'],
                    mode='lines',
                    name='val_loss'))

fig.update_layout(title='Loss',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')

fig.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.round(y_pred).astype(int)

cm = confusion_matrix(y_test, y_pred)
plot_confusion_matrix(cm=cm, classes=['stroke', 'no stroke'], title='Confusion Matrix')
